# Load Libraries

In [3]:
import snowflake.connector
import os
import numpy as np
import pandas as pd

# Connection to snowflake

In [4]:
def connec_to_sf(user):
    try:
        cnn = snowflake.connector.connect(
            user=user,
            authenticator='externalbrowser',
            role='',
            warehouse='',
            database='',
            schema='',
            account=''
        )
        cs = cnn.cursor()
        print('Connection successfull')
        return cs,cnn
    except: 
        print('Error connecting to Snowflake: Check your credentials and VPN connection')
        raise
    
user1 = os.environ['']
cs, cnn = connec_to_sf(user1)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://sso-dbbfec7f.sso.duosecurity.com/saml2/sp/DIO232UHAG8BJAFM294O/sso?SAMLRequest=lVJdb%2BIwEPwrke85sROggAVUKSmUExyohDuJN8cx1Gpi57xOU%2F59HT6k3kMr3Zs%2FZnZmd3Z0%2F14W3pswILUaozAgyBOK61yq4xjt0pk%2FQB5YpnJWaCXG6CQA3U9GwMqionFtX9Sz%2BFsLsJ4rpIC2H2NUG0U1AwlUsVIAtZxu49WSRgGhDEAY6%2BTQlZKDdFov1lYU46ZpgqYTaHPEESEEkyF2qBbyA32SqL7XqIy2muviRnl3PX0hEWLSbSUcwilsrsQHqS4j%2BE4lu4CAPqXpxt%2Bstyny4lt3U62gLoXZCvMmudg9Ly8GwDngErgOavAFA%2BuHASjdHAr2Krguq9q6moE74YPIcaGP0k1qkYxR9Spzvefz4WM63c%2Bzquzy%2FjFMSLrLprvekrCio6NVM13%2ByZIleeTI%2B33LNWpzXQDUYqHaNK17IlHHJ30%2FukvDO0r6tBcF4aC3R17i0pSK2TPzZhlA%2B3mWHQTvHwJ3CfJag%2BC1kfZ0NtumEmGocLJYR51o9xTPBw8%2F49kqGnbXLR1dtoaebZjJ%2F81ihD9zr%2Bv3yyWySDa6kPzkzbQpmf06sDAIzy8y9w9nKBUlk0Wc50YAuOCKQjdTI5h1W25NLRCeXFT%2F3f

# QUERY

In [5]:
query = """SELECT DISTINCT INCIDENT_NUMBER, "PARTY ID", "PARTY NAME", CURRENT_SEVERITY_INT, COMPLEXITY_DESCR, INCDT_CREATION_DATE, CLOSED_DATE, SR_TIME_TO_CLOSE_DAYS_CNT, OUTAGE_FLAG, BUG_CNT, ENTRY_CHANNEL_NAME, PROBLEM_CODE, RESOLUTION_CODE, REQUEST_TYPE_NAME, INCIDENT_CONTRACT_STATUS, SERVICE_PROGRAM, SERVICE_SUBGROUP_DESC, SERVICE_LEVLE_CODE, SERVICE_BRAND, SR_PRODUCT_ID, TAC_HW_PLATFORM_NAME, SR_TECHNOLOGY, TECH_NAME, SR_SUB_TECHNOLOGY, RU_BK_PRODUCT_FAMILY_ID, BE_INT, SUB_BE_INT, CUSTOMER_MARKET_SEGMENT_CD, ISO_COUNTRY_CD
FROM CX_DB.CX_CA_BR.TAC_UNION TU
LEFT JOIN CX_DB.CX_CA_BR.ACCOUNT_ID_LOOKUP AIL
ON TU."PARTY ID" = AIL.PARTY_ID
RIGHT JOIN CX_DB.CX_CA_BR.SUCCESS_TRACK_PRICING STP
ON TU.SR_PRODUCT_ID = STP.PRODUCT_SKU
WHERE AIL.SALES_LEVEL_2 = 'US COMMERCIAL' AND TU.INCDT_CREATION_DATE > '2020-06-07' """
 
cs.execute(query)
df = cs.fetch_pandas_all()
df.to_csv(r'C:\Users\carloma5\Extraction\raw_data.csv', index = False)

# DATA PREPARATION

## Data type coercion

In [6]:
df['INCDT_CREATION_DATE'] = pd.to_datetime(df['INCDT_CREATION_DATE'])
df['CLOSED_DATE'] = pd.to_datetime(df['CLOSED_DATE'])
df['BUG_CNT'] = df['BUG_CNT'].fillna(0) 

## Outliers removing

In [7]:
def remove_ouliers(df, column):
    data = df[column]
    Q1, Q3 = np.quantile(data, [0.25, 0.75])
    IQR = Q3-Q1
    outliers = (data<Q1-1.5*IQR) | (data>Q3+1.5*IQR)
    return df[~outliers].copy()

df = remove_ouliers(df, 'SR_TIME_TO_CLOSE_DAYS_CNT')

## Service program grouping

In [8]:
sspt_list= ['3Y SSPT PLUS', '3YR ETSP', '5Y SSPT PLUS', 'SSPT EXPRESS', 'SSPT PLUS', 'SSPT PLUS DR', 'SSPT PLUS2', 
            'SSPT SW SUB', 'SP SSPT OPE', 'SP SSPT PLUS', 'SSPL BASIC', 'SSPL BASIC', 'TS SW PREF']

sntc_list= ['SNTC', '3YR SMARTNET', '5YR SMARTNET', 'LL UCS DR', 'ONSITE UCS', 'UC SUP 3YR', 'UC SUP DR 3Y', 
            'ESHARDWARE', 'UCS DR', 'UCS HW ONLY', 'UCS HW+DR', 'SIA HW SPT', 'SIA SW B', 'COSE-TELPRES', 'SPBASE']

cx_list= ['CX L1 3YR', 'CX L1 5YR', 'CX L1 DCN', 'CX L2 3YR', 'CX L2 5YR', 'CX LEVEL 1', 'CX LEVEL 2', 'CX LEVEL 2 DR', 
          'CXEA T1HWSPT']

swss_list= ['PSS SWSS', 'PSS UCC SWSS', 'CISCO SWSS', 'SWSS 3Y DIST', 'SWSS CLOUD', 'SWSS HCS', 'ELA SWSS', 
            'PSS SWSS', 'PSS UCC SWSS', 'ELA SWSS', 'SSPLS SWSS B', 'SS PLUS SWSS', 'ENH ADD ON', 'SWAPPS', 'UC ISV']

partner_list= ['PLSS', 'PRTNR SS ROW', 'PRTNR SS USC', 'PRTNR SUP', 'PRTNR TP VID', 'PSS UCS DR', 'PSS UCS SUPP', 
               'PSS UCS WP']

transactional_list= ['SMB SA', 'CSE(SUP ESS)', 'ETSP', 'NC (AS-TS)', 'NC(AS-TS)11', 'NC(AS-TS)3', 'NC(AS-TS)30', 
                     'NC(AS-TS)31', 'NC(AS-TS)32', 'NC(AS-TS)33', 'NC(AS-TS)38', 'NC(AS-TS)39', 'NC(AS-TS)5', 
                     'NC(AS-TS)9', 'CIPS', 'EXT WARRANTY']

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(sspt_list)]= 'SSPT'

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(sntc_list)]= 'SNTC'

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(cx_list)]= 'CX'

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(swss_list)]= 'SWSS'

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(partner_list)]= 'PARTNER_SUPPORT'

df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(transactional_list)]= 'TRANSACTIONAL'

C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\3478307228.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(sspt_list)]= 'SSPT'
C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\3478307228.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICE_PROGRAM'][df['SERVICE_PROGRAM'].isin(sntc_list)]= 'SNTC'
C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\3478307228.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

## Outage flag grouping

In [9]:
yes_list= ['Yes', 'TRUE', 'true'] 
no_list= ['No', 'XYZ', 'FALSE', 'false']

df['OUTAGE_FLAG'][df['OUTAGE_FLAG'].isin(yes_list)]= '1'

df['OUTAGE_FLAG'][df['OUTAGE_FLAG'].isin(no_list)]= '0'

C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\2024651149.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OUTAGE_FLAG'][df['OUTAGE_FLAG'].isin(yes_list)]= '1'
C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\2024651149.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OUTAGE_FLAG'][df['OUTAGE_FLAG'].isin(no_list)]= '0'


## Resolution code grouping

In [10]:
cust_edu_list= ['CUSTOMER ACTIVATION', 'CUSTOMER_EDUCATION']

doc_provided_list= ['CONTENT DELIVERED', 'DOCUMENTATION PROVIDED']

rma_list= ['HW_REPLACE_SVO']

sw_config_list= ['SW_CONFIG', 'SW_UPGRD_EXSTNG_DEFECT', 'SW_UPGRD_NEW_DEFECT', 'SW_UPGRD_NEW_FEATURE']

third_party_list= ['REDIRECT_3RD_PARTY']

others_list= ['APPLICATION ERROR - MAC', 'APPLICATION ERROR - WPC', 'CLOSED_NOT_ENTITLED', 'CONTENT NOT AVAILABLE', 
              'CONTENT REQUEST', 'DUPLICATE', 'HW_UPGRD_ EXSTNG_DEFECT', 'HW_UPGRD_ NEW_DEFECT', 'HW_UPGRD_NEW_FEATURE',
              'LICENSE_ISSUE', 'MOBILE - IOS', 'MOBILE - ANDRIOD', 'NETWORK_REDESIGN', 'NEW FEATURE REQUEST',
              'NEW USER ACCESS', 'NO_RESPONSE', 'NOT AVAILABLE', 'OPENED_IN_ERROR', 'PARTNER_SOLVED',
              'REDIRECT_PARTNER_RESELL', 'REPORTS', 'RESET PASSWORD', 'RESOLV_BY_CUST', 'SANDBOX UPLOAD',
              'UNREPRODUCIBLE', 'UNRESOLVED', 'VIEWER - ONLINE']

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(cust_edu_list)]= 'CUST EDU'

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(doc_provided_list)]= 'DOC PROVIDED'

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(rma_list)]= 'RMA'

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(sw_config_list)]= 'SW CONFIG'

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(third_party_list)]= '3RD PARTY'

df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(others_list)]= 'OTHER'

C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\1520275138.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(cust_edu_list)]= 'CUST EDU'
C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\1520275138.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RESOLUTION_CODE'][df['RESOLUTION_CODE'].isin(doc_provided_list)]= 'DOC PROVIDED'
C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\1520275138.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

## Renaming of the columns

In [11]:
new_headers= {
    'PARTY ID': 'PARTY_ID',
    'PARTY NAME': 'PARTY_NUMBER',
    'CURRENT_SEVERITY_INT': 'SEVERITY',
    'COMPLEXITY_DESCR': 'COMPLEXITY',
    'INCDT_CREATION_DATE': 'CREATION_DATE',
    'CLOSED_DATE': 'CLOSED_DATE',
    'SR_TIME_TO_CLOSE_DAYS_CNT': 'TIME_TO_CLOSE_DAYS',
    'OUTAGE_FLAG': 'OUTAGE',
    'BUG_CNT': 'BUG_COUNT',
    'ENTRY_CHANNEL_NAME': 'COMMUNICATION_CHANNEL',
    'PROBLEM_CODE': 'PROBLEM_CODE',
    'RESOLUTION_CODE': 'RESOLUTION_CODE',
    'REQUEST_TYPE_NAME': 'REQUEST_TYPE',
    'INCIDENT_CONTRACT_STATUS': 'CONTRACT_STATUS',
    'SERVICE_PROGRAM': 'SERVICE',
    'SERVICE_SUBGROUP_DESC': 'SERVICE_SUBGROUP',
    'SERVICE_LEVLE_CODE': 'SERVICE_LEVEL',
    'SERVICE_BRAND': 'SERVICE_BRAND',
    'SR_PRODUCT_ID': 'PRODUCT_ID',
    'TAC_HW_PLATFORM_NAME': 'DEVICE_NAME',
    'SR_TECHNOLOGY': 'TECHNOLOGY_1',
    'TECH_NAME': 'TECHNOLOGY_2',
    'SR_SUB_TECHNOLOGY': 'SUB_TECHNOLOGY',
    'RU_BK_PRODUCT_FAMILY_ID': 'PRODUCT_FAMILY',
    'BE_INT': 'ARCHITECTURE',
    'SUB_BE_INT': 'SUB_ARCHITECTURE',
    'CUSTOMER_MARKET_SEGMENT_CD': 'MARKET_SEGMENT',
    'ISO_COUNTRY_CD': 'COUNTRY'
}

df= df.rename(columns= new_headers) 

## Architectures grouping

In [12]:
df['ARCH_GROUPING']= np.nan

Campus = ['Enterprise Routing', 'Enterprise Switching', 'Wireless', 'Service Provider Routing']

Datacenter_Switching = ['Cloud Networking']

Datacenter_Computing = ['Cloud and Compute']

IOT = ['IOT']

Security = ['Security']

df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(Campus)]= 'Campus'

df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(Datacenter_Switching)]= 'Datacenter Switching'

df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(Datacenter_Computing)]= 'Datacenter Computing'

df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(IOT)]= 'IOT'

df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(Security)]= 'Security'

C:\Users\carloma5\AppData\Local\Temp\ipykernel_13140\586529724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ARCH_GROUPING'][df['ARCHITECTURE'].isin(Campus)]= 'Campus'


In [13]:
df.to_csv(r'C:\Users\carloma5\Processing\data.csv', index = False)